In [1]:
import torch
from src.training.regression_task import RegressionTask
from src.networks.generic_sfcn import ScriptableGenericSFCNModel
from src import config

/opt/anaconda/envs/bowl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
best = (
    RegressionTask.load_from_checkpoint("article/models/SFCN_conv2_aug_shift.ckpt")
    .eval()
    .cpu()
)
best.model = best.model._orig_mod

scriptable = ScriptableGenericSFCNModel(best.model.hp)
scriptable.load_state_dict(best.model.state_dict(), strict=False)
best.model = scriptable

script = torch.jit.freeze(torch.jit.script(best.model.eval()))

In [13]:
with torch.inference_mode():
    rand_data = torch.rand(
        (
            1,
            1,
        )
        + config.VOLUME_SHAPE,
        requires_grad=False,
    )
    print(script(rand_data))

tensor([[1.2066e-03, 5.0905e-03, 1.6413e-02, 3.8539e-02, 6.6410e-02, 8.7233e-02,
         9.3568e-02, 9.2545e-02, 9.1266e-02, 9.0103e-02, 8.6779e-02, 7.7945e-02,
         6.2769e-02, 4.3900e-02, 2.7018e-02, 1.5702e-02, 9.1812e-03, 5.5979e-03,
         3.6461e-03, 2.6208e-03, 2.1225e-03, 1.8695e-03, 1.7022e-03, 1.5880e-03,
         1.5614e-03, 1.6464e-03, 1.8000e-03, 1.9812e-03, 2.2567e-03, 2.6709e-03,
         3.1136e-03, 3.4003e-03, 3.5345e-03, 3.6328e-03, 3.6634e-03, 3.6149e-03,
         3.6055e-03, 3.7339e-03, 4.0407e-03, 4.3993e-03, 4.6418e-03, 4.8156e-03,
         4.8591e-03, 4.5062e-03, 3.6389e-03, 2.3884e-03, 1.1747e-03, 4.0190e-04,
         8.9050e-05, 1.1876e-05]])


In [ ]:
torch.jit.save(script, "article/model_scripted/best_torchscript.pt")

In [16]:
with torch.inference_mode():
    rand_data = torch.rand(
        (
            1,
            1,
        )
        + config.VOLUME_SHAPE,
        requires_grad=False,
    )
    print(torch.jit.load("article/model_scripted/best_torchscript.pt")(rand_data))

tensor([[7.0513e-04, 2.8634e-03, 8.9661e-03, 2.0598e-02, 3.5607e-02, 4.9687e-02,
         6.0194e-02, 6.8304e-02, 7.6481e-02, 8.5037e-02, 9.2118e-02, 9.3506e-02,
         8.5791e-02, 7.0080e-02, 5.1636e-02, 3.5816e-02, 2.4445e-02, 1.6771e-02,
         1.1723e-02, 8.6049e-03, 6.8131e-03, 5.7360e-03, 4.9701e-03, 4.3807e-03,
         3.9860e-03, 3.8021e-03, 3.7346e-03, 3.7008e-03, 3.7783e-03, 3.9902e-03,
         4.1811e-03, 4.1834e-03, 4.0591e-03, 3.9211e-03, 3.7185e-03, 3.4621e-03,
         3.2763e-03, 3.2325e-03, 3.3388e-03, 3.4804e-03, 3.5410e-03, 3.5714e-03,
         3.5281e-03, 3.2234e-03, 2.5783e-03, 1.6855e-03, 8.3180e-04, 2.8834e-04,
         6.5585e-05, 9.1502e-06]])
